# Message Passing
Prerequisite : Understand functions as if they are **values** (i.e.: first-class citizen)
# Represent Rational Number

Beside using `class` keyword, we can represent the Rational data type using **message passing style** like below.

In [61]:
def Rational(n, d):
    assert(d != 0)
    from math import gcd
    g = gcd(n,d)
    def dispatch(msg):
        if msg == "numer":
            return n // g
        elif msg == "denom":
            return d // g
        else:
            raise Exception(f"{msg} : Unknown message for Rational instance")
    return dispatch

def numer(r):
    return r("numer")

def denom(r):
    return r("denom")

def rational_str(r):
    return f"Rational({numer(r)}, {denom(r)})"

def is_eq_ratio(r1, r2):
    return numer(r1)*denom(r2) == denom(r1)*numer(r2)

def add_ratio(r1, r2):
    return Rational(
        numer(r1)*denom(r2) + numer(r2)*denom(r1),
        denom(r1)*denom(r2))

def sub_ratio(r1, r2):
    return Rational(
        numer(r1)*denom(r2) - numer(r2)*denom(r1),
        denom(r1)*denom(r2))

def mul_ratio(r1, r2):
    return Rational(
        numer(r1) * numer(r2),
        denom(r1) * denom(r2))

def div_ratio(r1, r2):
    return Rational(
        numer(r1) * denom(r2),
        denom(r1) * numer(r2))

def test_rational():
    r1 = Rational(1,2)
    r2 = Rational(3,4)
    ans = Rational(5,4)
    assert(is_eq_ratio(add_ratio(r1,r2), ans))
    ans = Rational(-1,4)
    assert(is_eq_ratio(sub_ratio(r1,r2), ans))
    ans = Rational(2,3)
    assert(is_eq_ratio(div_ratio(r1,r2), ans))
    ans = Rational(3,8)
    assert(is_eq_ratio(mul_ratio(r1,r2), ans))

test_rational()

In [62]:
r1 = Rational(1,2)
r2 = Rational(3,4)
print_ratio = lambda r: print(rational_str(r))
print_ratio(r1)
print_ratio(r2)
print_ratio(add_ratio(r1,r2))
print_ratio(sub_ratio(r1,r2))
print_ratio(div_ratio(r1,r2))
print_ratio(mul_ratio(r1,r2))

Rational(1, 2)
Rational(3, 4)
Rational(5, 4)
Rational(-1, 4)
Rational(2, 3)
Rational(3, 8)


# Represent Pair

In [63]:
def Pair(x,y):
    def dispatch(msg):
        if msg == "fst":
            return x
        elif msg == "snd":
            return y
        elif msg == "repr":
            return f"Pair({x},{y})"
        elif msg == "print":
            print(dispatch("repr"))
            return dispatch("repr")
        else:
            raise Exception(f"{msg}: Unknown message for Pair instance") 
    return dispatch

def fst(p):
    return p("fst")

def snd(p):
    return p("snd")

In [64]:
p1 = Pair(9, "Cirno")
print(fst(p1), snd(p1))
p1("print")

9 Cirno
Pair(9,Cirno)


'Pair(9,Cirno)'

# Change underlying representation of Rational

In [66]:
def Rational(n, d):
    assert(d != 0)
    from math import gcd
    g = gcd(n,d)
    return Pair(n // g, d // g)

def numer(r):
    return r("fst")

def denom(r):
    return r("snd")

test_rational()

The Rational program is changed to use `Pair` rather using message passing style alone. However, other part of program like `add_rational` does not need modification. 
This is an independence between underlying representation and upper system. It is achieved through defining selector. 

In [69]:
def Rational(n, d):
    assert(d != 0)
    from math import gcd
    g = gcd(n,d)
    return n // g, d // g # this return a tuple

def numer(r):
    return r[0]

def denom(r):
    return r[1]

test_rational() # check that if it works as expected

# What is data?

You maybe suprised that the data structure `Pair` is not even a value but it is a function. So is the `Rational`. Nevetheless, the `Rational` is working same as using `class` keyword in `Python`. Indeed, there is no much difference between data and procedure.

In [9]:
p = Pair(1,2)
p

<function __main__.Pair.<locals>.dispatch(msg)>

# Represent Bank Account, `nonlocal`, lexical closure

To model the real world, we often perceive the world composed of many independent objects. Objects are independent and each has **states** that change overtime. An object is said to "have state" if its behavior is influnced by its history. Bank account is an abstract entity but its balance changes over time hence it can be an object.

In [17]:
def withdraw(acc, amt):
    if amt > acc:
        return "insufficient fund"
    return acc - amt
meiling_acc = 100  
print(meiling_acc)
meiling_acc =  withdraw(meiling_acc, 25)
meiling_acc

100


75

Above is rather cumbersome that reassignment is required to update the balance (state). Code below is done using lexical closure.

In [18]:
def new_account(balance):
    def withdraw(amt):
        nonlocal balance
        if amt > balance:
            return "insufficient fund"
        balance = balance - amt
        return balance
    return withdraw

In [19]:
sakuya_acc = new_account(125)
print(sakuya_acc(25))
print(sakuya_acc(75))
print(sakuya_acc(30))

100
25
insufficient fund


To include the `deposit` operation, we can use message passing style to represent Bank Account

In [21]:
def make_account(balance):
    def withdraw(amt):
        nonlocal balance
        if amt > balance:
            return "insufficient fund"
        balance = balance - amt
        return balance
    def deposit(amt):
        if amt < 0:
            return f"{amt}, no negative deposit transaction"
        nonlocal balance
        balance = balance + amt
        return balance
    def dispatch(msg):
        if msg == "withdraw":
            return withdraw
        elif msg == "deposit":
            return deposit
        else:
            raise Exception(f"{msg}: Unknown message for Bank Account Instance")
    return dispatch

In [23]:
rumia_acc = make_account(75)
print(rumia_acc("deposit")(35))
print(rumia_acc("withdraw")(25))

110
85


## Side Note: scoping, first-class functions, lexical closure

In [10]:
pi = 3.14
def sphere_volume(r):
    return 4*r**3*pi/3
print(sphere_volume(3))
from math import pi
print(sphere_volume(3))

113.04
113.09733552923255


In the function `sphere_volume`, programming languages must somehow to resolve the value of the variable `pi`.

Additinoally, a function is also an enclosing environment over its parameters and variables (ie.: these variables and parameters are local to the function). 
I would say these variables and parameters are in inner scope.
Outer scope programs cannot directly assign the value of inner scoped variables.

```
>>> def sphere_volume(r):
...     pi = 3.14
...     return 4*r**3*pi/3
...
>>> print(pi)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'pi' is not defined
```

To resolve the value of variable, if it is not defined in inner scope, then search in its outer scope, and repeat until either it is defined or reach global scope. This is known as lexical scoping.

## Side note to Scoping
The concept of scoping is quite natural to who learn the algebra and basic of mathematical function. For example, below is general form of linear function.

$$
f(x) = mx + h
$$

Sometimes, the classes do not explicitly mention where the $x$, $m$ and $h$ variables should look for their values. Then, students somehow know that the function value depend on the value of $x$, if $m$ and $h$ are not given.

In field of logic study, the $x$ is said to be bound by the function, whereas $m$ and $h$ are said to be free. The $x$ is called *bound variable* while $m$ and $h$ are called *free variable*.

In [5]:
def compose(f,g):
    def fg(x):
        return f(g(x))
    return fg
inc = lambda x : x + 1
sqr = lambda x : x * x
dec = lambda x : x - 1
inc_sqr = compose(inc, sqr)
dec_sqr = compose(dec, sqr)
inc_sqr(7), dec_sqr(7)

(50, 48)

in `Python`, functions are first-class citizen such that it is allowed to:
1. pass functions as arguments
2. return functions as values
3. assign functinos as values
4. like values, can be included in other data structures

When the function `compose` is called, it returns a function `fg` which involves other 2 functions `f`, and `g`. The question is that which values `f` and `g` refer to? 

By lexical closure, function is also an enclosing environment which keep track the values of variables.

For example, `inc_sqr` maintains the record of `f = inc` and `g = dec`. The `dec_sqr` maintains the record of `f = dec` and `g = dec`

```
>>> def counter(start = 0):
...     def next():
...         start += 1
...         return start
...     return next
...
>>> i = counter(5)
>>> i()
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 3, in next
UnboundLocalError: local variable 'start' referenced before assignment
```

Above code is illegal in `Python`. `Python` default does not support full lexical closure. To fix this, we can use the keyword `nonlocal`. Beside that, we may define `generator` to support similar functionality.

In [18]:
def counter(start = 0):
    start -= 1
    def next():
        nonlocal start
        start += 1
        return start
    return next

def count(start = 0):
    while True:
        yield start
        start += 1

i = counter(5)
j = count(5)
print(i(), next(j))
print(i(), next(j))
print(i(), next(j))
print(i(), next(j))

5 5
6 6
7 7
8 8
